In [25]:
import pandas as pd
import numpy as np
import json

In [26]:

LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal", 
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

# json으로 변환할 csv 파일 경로 및 저장 위치 설정              
SUBMISSION_CSV = './submission.csv'
CONVERTED_JSON = './submission_coco.json'

submission_df = pd.read_csv(SUBMISSION_CSV)

In [29]:
global annid
annid = 0
images = []
categories = []
annotations = []

submission_df['fileid'] = submission_df['image_id'].astype('category').cat.codes

def image(row):
    image = {}
    image["width"] = 1024
    image["height"] = 1024
    image["file_name"] = row.image_id
    image["id"] = row.fileid
    return image

def category(i, label_name):
    category = {}
    category["id"] = i
    category["name"] = label_name
    category["supercategory"] = label_name
    return category

# boxes == [xmin, ymin, xmax, ymax]
def annotation(row):
    global annid
    annotations = []

    boxes = np.array(row.PredictionString.split(' '))
    if len(boxes) % 6 == 1:
        boxes = boxes[:-1].reshape(-1, 6)
    elif len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    else:
        raise Exception('error', 'invalid box count')
    for box in boxes:
        xmin, xmax = float(box[2]), float(box[4])
        ymin, ymax = float(box[3]), float(box[5])
    
        annotation = {}
        area = (xmax - xmin)*(ymax - ymin)
        annotation["image_id"] = row.fileid
        annotation["category_id"] = int(box[0])
        annotation["score"] = float(box[1])
        annotation["area"] = area
        annotation["bbox"] = [xmin, ymin, xmax-xmin, ymax-ymin]
        annotation["iscrowd"] = 0
        annotation["id"] = annid
        annotations.append(annotation)
        annid += 1
    return annotations

for row in submission_df.itertuples():
    annotations.extend(annotation(row))

for row in submission_df.itertuples():
    images.append(image(row))

for i, label_name in enumerate(LABEL_NAME):
    categories.append(category(i, label_name))

data_coco = {}
data_coco["images"] = images
data_coco["categories"] = categories
data_coco["annotations"] = annotations
json.dump(data_coco, open(CONVERTED_JSON, "w"), indent=4)